In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

import warnings
warnings.simplefilter(action='ignore')

In [2]:
test_df = pd.read_csv('test.csv', index_col=0)
train_df = pd.read_csv('train.csv', index_col=0)
code_d = pd.read_csv('D_code.csv')
code_h = pd.read_csv('H_code.csv')
code_l = pd.read_csv('L_code.csv')
submission_df = pd.read_csv('sample_submission.csv')
code_d.drop(labels='Unnamed: 5', axis=1, inplace=True)

In [3]:
for i in ['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn', 'h_m_match_yn', 'h_s_match_yn'] :
    train_df[i] = train_df[i].astype('int')
    test_df[i] = test_df[i].astype('int')

In [4]:
# target 비율
train_df.target.value_counts(sort=False)/len(train_df)

0    0.50026
1    0.49974
Name: target, dtype: float64

In [5]:
# null 수
sum(train_df[train_df.isna()].count()) # 0
sum(test_df[test_df.isna()].count())   # 0

0

In [107]:
train_person_df = train_df.iloc[:,7:19]
train_content_df = train_df.iloc[:,19:30]
test_person_df = test_df.iloc[:,7:19]
test_content_df = test_df.iloc[:,19:30]

# random forest baseline 0.62704

In [7]:
# from sklearn.ensemble import RandomForestClassifier

# train_x = train_df.iloc[:,:30]
# train_y = train_df.iloc[:,[-1]]
# test_x = test_df.iloc[:,:30]
# test_y = test_df.iloc[:,[-1]]

# rf = RandomForestClassifier(max_depth=20)
# rf.fit(train_x, train_y)
# print(rf.score(train_x, train_y))  # 0.8726947451046019
# pred_y = rf.predict(test_x)

# submission_df['target'] = pred_y
# # submission_df.to_csv('sub1.csv', index=False)

In [9]:
code_d.columns = ['d_code', 'd_section_code', 'd_small_code', 'd_midium_code', 'd_large_code']
code_l.columns = ['l_code', 'l_section_code', 'l_small_code', 'l_midium_code', 'l_large_code']
code_h.columns = ['h_code', 'h_upper_code']

In [115]:
d = ['person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3']
h = ['person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3']

person_res_d = pd.DataFrame()
person_res_h = pd.DataFrame()

for idx in tqdm(range(len(test_person_df))):
    tmp_d = pd.DataFrame()
    tmp_h = pd.DataFrame()
    for col in d :  # ['person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3']  #  ['contents_attribute_d']
        tmp_code_num = test_person_df[col][idx]
        t = code_d[code_d['d_code']==tmp_code_num]
        t.drop('d_code', axis=1, inplace=True)
        t.index = [idx]
        t = t.add_prefix(col)
        tmp_d = pd.concat([tmp_d, t], axis=1)
    person_res_d = person_res_d.append(tmp_d, ignore_index=True)
    
    for col in h :  # ['person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3']  #  ['contents_attribute_d']
        tmp_code_num = test_person_df[col][idx]
        t = code_h[code_h['h_code']==tmp_code_num]
        t.drop('h_code', axis=1, inplace=True)
        t.index = [idx]
        t = t.add_prefix(col)
        tmp_h = pd.concat([tmp_h, t], axis=1)
    person_res_h = person_res_h.append(tmp_h, ignore_index=True)
    
    
# res.to_csv('person_d_code_detail.csv', index=False)

  0%|          | 0/46404 [00:00<?, ?it/s]

In [120]:
d = ['contents_attribute_d']
h = ['contents_attribute_h']
l = ['contents_attribute_l']

content_res_d = pd.DataFrame()
content_res_h = pd.DataFrame()
content_res_l = pd.DataFrame()

for idx in tqdm(range(len(test_content_df))):
    tmp_d = pd.DataFrame()
    tmp_h = pd.DataFrame()
    tmp_l = pd.DataFrame()
    for col in d :  # ['contents_attribute_d']
        tmp_code_num = test_content_df[col][idx]
        t = code_d[code_d['d_code']==tmp_code_num]
        t.drop('d_code', axis=1, inplace=True)
        t.index = [idx]
        t = t.add_prefix(col)
        tmp_d = pd.concat([tmp_d, t], axis=1)
    content_res_d = content_res_d.append(tmp_d, ignore_index=True)
    
    for col in h :
        tmp_code_num = test_content_df[col][idx]
        t = code_h[code_h['h_code']==tmp_code_num]
        t.drop('h_code', axis=1, inplace=True)
        t.index = [idx]
        t = t.add_prefix(col)
        tmp_h = pd.concat([tmp_h, t], axis=1)
    content_res_h = content_res_h.append(tmp_h, ignore_index=True)
    
    for col in l : 
        tmp_code_num = test_content_df[col][idx]
        t = code_l[code_l['l_code']==tmp_code_num]
        t.drop('l_code', axis=1, inplace=True)
        t.index = [idx]
        t = t.add_prefix(col)
        tmp_l = pd.concat([tmp_l, t], axis=1)
    content_res_l = content_res_l.append(tmp_l, ignore_index=True)
    
# content_res_l.to_csv('content_l_code_detail.csv', index=False)
# content_res_h.to_csv('content_h_code_detail.csv', index=False)
# content_res_d.to_csv('content_d_code_detail.csv', index=False)

  0%|          | 0/46404 [00:00<?, ?it/s]

In [138]:
new_test = pd.concat([test_person_df, person_res_d, person_res_h, test_content_df, content_res_d, content_res_h, content_res_l], axis=1)
new_test_y = test_df.iloc[:,[-1]]
new_test

,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,...,contents_attribute_h,contents_attribute_dd_section_code,contents_attribute_dd_small_code,contents_attribute_dd_midium_code,contents_attribute_dd_large_code,contents_attribute_hh_upper_code,contents_attribute_ll_section_code,contents_attribute_ll_small_code,contents_attribute_ll_midium_code,contents_attribute_ll_large_code
0,1,2,1,857,851,1227,4,1,1,263,...,263,836,824,831,744,250,1146,1128,1021,2010
1,0,2,2,683,1086,662,2,1,1,258,...,263,276,274,274,216,250,1610,1606,1605,2016
2,3,2,1,514,790,1233,0,1,1,177,...,177,490,482,490,482,169,1812,1811,1810,2020
3,2,2,5,114,181,175,4,1,1,177,...,177,145,56,123,1,169,101,100,99,2006
4,6,4,5,1082,1078,1056,5,1,1,178,...,177,1097,1093,1094,926,169,984,980,954,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46399,0,4,1,147,46,145,4,1,1,2,...,91,145,56,123,1,85,759,759,759,2006
46400,0,4,1,176,120,159,4,1,1,86,...,91,145,56,123,1,85,759,759,759,2006
46401,0,3,1,145,46,147,5,1,1,288,...,288,145,56,123,1,277,759,759,759,2006
46402,3,3,2,145,46,147,6,1,1,288,...,288,145,56,123,1,277,759,759,759,2006


In [110]:
new_train = pd.concat([train_person_df, train_content_df], axis=1)
new_train_y = train_df.iloc[:,[-1]]
new_train

,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,...,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h
0,4,3,5,275,370,369,8,1,1,4,...,3,10,2,1,2,1608,275,1,4,139
1,3,4,1,114,181,175,4,1,1,131,...,3,5,1,1,2,1608,275,1,4,133
2,0,3,5,464,175,452,3,1,1,54,...,1,10,2,1,1,1600,94,1,4,53
3,0,2,5,703,705,704,3,1,1,72,...,3,5,1,1,2,1608,275,5,3,74
4,3,4,5,275,370,369,4,1,1,214,...,1,10,2,1,2,1608,275,1,4,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501946,1,2,2,1192,935,1228,3,1,1,59,...,3,5,1,1,2,354,147,1,5,65
501947,6,2,1,118,113,110,4,1,1,105,...,3,10,2,1,2,163,120,1,4,142
501948,7,4,1,147,46,145,4,1,1,59,...,1,5,1,1,2,438,147,2,7,65
501949,1,2,1,46,147,145,4,1,1,251,...,2,5,1,1,2,660,147,3,4,259


In [158]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=20)
rf.fit(new_train, new_train_y)
print(rf.score(new_train, new_train_y))  # 0.8610222910204383
# pred_y = rf.predict(test_x)

# submission_df['target'] = pred_y
# submission_df.to_csv('sub1.csv', index=False)

0.8515831226553986


In [159]:
pred_y = rf.predict(new_test)

submission_df['target'] = pred_y
submission_df.to_csv('sub3.csv', index=False)

In [ ]:
for i in ['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn', 'h_m_match_yn', 'h_s_match_yn'] :
    plt.figure(figsize=(10,5))
    plt.title(i, fontsize=15)
    sns.countplot(data=train_df, x=i, hue='target')